In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 970.4/970.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 18.2 MB/s eta 0:00:00


In [3]:
import os
import json
import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en', processors='tokenize,pos,lemma,ner') # initialize English neural pipeline

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| pos       | combined_charlm           |
| lemma     | combined_nocharlm         |
| ner       | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [4]:
# Global variable to store all extracted texts
extracted_texts = []

In [5]:
def process_text_with_stanza(text):
    doc = nlp(text)
    return doc


In [15]:
def extract_info_from_json(json_file_path, hadm_id_set):
    global extracted_texts

    # Open the JSON file
    with open(json_file_path, 'r') as file:
        # Load JSON data from the file
        data = json.load(file)

        # Extract 'hadm_id' and 'comment' from the JSON data
        hadm_id = data.get('hadm_id', None)
        comment = data.get('comment', None)

        # If 'hadm_id' is not found, print a warning message
        if hadm_id is None:
            print(f"Warning: 'hadm_id' not found in {json_file_path}")
            return None

        # Add 'hadm_id' to the set
        hadm_id_set.add(hadm_id)

        # Extract 'notes' from the JSON data
        notes = data.get('notes', [])

        # Iterate through each note
        for note in notes:
            note_info = {}  # Initialize a dictionary to store note information

            # Add 'hadm_id' to the note information
            note_info['hadm_id'] = hadm_id

            # Extract information from the note
            note_info['note_id'] = note.get('note_id', None)
            note_info['category'] = note.get('category', None)
            note_info['description'] = note.get('description', None)

            # Extract annotations from the note
            annotations = note.get('annotations', [])
            annotations_info = []  # Initialize a list to store annotation information

            # Iterate through each annotation in the note
            for annotation in annotations:
                annotation_info = {}  # Initialize a dictionary to store annotation information

                # Extract information from the annotation
                annotation_info['begin'] = annotation.get('begin', None)
                annotation_info['end'] = annotation.get('end', None)
                annotation_info['code'] = annotation.get('code', None)
                annotation_info['code_system'] = annotation.get('code_system', None)
                annotation_info['description'] = annotation.get('description', None)
                annotation_info['type'] = annotation.get('type', None)
                annotation_info['covered_text'] = annotation.get('covered_text', None)

                annotations_info.append(annotation_info)  # Append annotation information to the list

            note_info['annotations'] = annotations_info  # Add annotations information to the note
            note_info['text'] = note.get('text', None)

            # Process text with Stanza
            if note_info['text']:
                processed_text = process_text_with_stanza(note_info['text'])
                note_info['processed_text'] = processed_text

                # Count number of sentences and store it
                num_sentences = len(processed_text.sentences)
                note_info['num_sentences'] = num_sentences

            extracted_texts.append(note_info)  # Append note information to the global variable

In [16]:
def search_files(folder_path):
    # Set to store unique hadm_id values
    hadm_id_set = set()

    # Recursively search for JSON files in the folder and its subfolders
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith('.json'):
                # Construct the full path to the JSON file
                json_file_path = os.path.join(root, filename)
                print("Processing:", json_file_path)
                # Call extract_info_from_json function to extract information from the JSON file
                extract_info_from_json(json_file_path, hadm_id_set)
                # Print a separator after processing each file
                print("=" * 50)

    # Print the count of unique hadm_id values
    print("Total unique hadm_id count:", len(hadm_id_set))


In [17]:
# Specify the path to the main folder containing subfolders with JSON files
main_folder_path = "/content/drive/MyDrive/#medical coding/with_text/gold"

# Call the search_files function to start searching for JSON files in the main folder and its subfolders
search_files(main_folder_path)

Processing: /content/drive/MyDrive/#medical coding/with_text/gold/Profee/ICD-10/1.0/187213-ICD-10.json
Processing: /content/drive/MyDrive/#medical coding/with_text/gold/Profee/ICD-10/1.0/196522-ICD-10.json
Processing: /content/drive/MyDrive/#medical coding/with_text/gold/Profee/ICD-10/1.0/193800-ICD-10.json
Processing: /content/drive/MyDrive/#medical coding/with_text/gold/Profee/ICD-10/1.0/162131-ICD-10.json
Processing: /content/drive/MyDrive/#medical coding/with_text/gold/Profee/ICD-10/1.0/142289-ICD-10.json
Processing: /content/drive/MyDrive/#medical coding/with_text/gold/Profee/ICD-10/1.0/128511-ICD-10.json
Processing: /content/drive/MyDrive/#medical coding/with_text/gold/Profee/ICD-10/1.0/142321-ICD-10.json
Processing: /content/drive/MyDrive/#medical coding/with_text/gold/Profee/ICD-10/1.0/190860-ICD-10.json
Processing: /content/drive/MyDrive/#medical coding/with_text/gold/Profee/ICD-10/1.0/192435-ICD-10.json
Processing: /content/drive/MyDrive/#medical coding/with_text/gold/Profee/

In [18]:
# # We can access named entities from 'processed_text' field of each note
# for note_info in extracted_texts:
#     if 'processed_text' in note_info:
#         processed_text = note_info['processed_text']
#         for sentence in processed_text.sentences:
#             for entity in sentence.ents:
#                 print(f"Entity: {entity.text}\tType: {entity.type}")

In [19]:
# Print the first sentence and its associated hadm_id
#for note_info in extracted_texts:
    #if 'processed_text' in note_info:
        #processed_text = note_info['processed_text']
        # Extract the first sentence
        #first_sentence = processed_text.sentences[0].text
        # Print the first sentence and its associated hadm_id
        #print(f"HADM_ID: {note_info['hadm_id']} - First Sentence: {first_sentence}")
        # num_sentences = len(processed_text.sentences)
        # print(f"HADM_ID: {note_info['hadm_id']} - Number of Sentences: {num_sentences}")

In [20]:
# Calculate total number of sentences
total_sentences = sum(note_info['num_sentences'] for note_info in extracted_texts if 'num_sentences' in note_info)

# Print total number of sentences
print("Total number of sentences:", total_sentences)

Total number of sentences: 156610


In [21]:
pip install transformers

In [22]:
from stanza.utils.conll import CoNLL

def tokenize_sentences(doc):

    tokenized_sentences = []  # Initialize an empty list to store tokenized sentences
    for sentence in doc.sentences:  # Iterate through each sentence in the document
        tokens = [token.text for token in sentence.tokens]  # Extract text of each token in the sentence
        tokenized_sentences.append(tokens)  # Append the list of tokens to the tokenized_sentences list
    return tokenized_sentences  # Return the tokenized sentences

# Loop through each note_info in the extracted_texts list
for note_info in extracted_texts:
    # Check if the 'processed_text' key exists in the current note_info
    if 'processed_text' in note_info:
        processed_text = note_info['processed_text']  # Extract the processed text from note_info
        tokenized_sentences = tokenize_sentences(processed_text)  # Tokenize the sentences in the processed text
        note_info['tokenized_sentences'] = tokenized_sentences  # Add the tokenized sentences to note_info

# Loop through each note_info in the extracted_texts list
for note_info in extracted_texts:
    # Check if the 'tokenized_sentences' key exists in the current note_info
    if 'tokenized_sentences' in note_info:
        print(f"Note ID: {note_info['note_id']}")  # Print the note ID
        tokenized_sentences = note_info['tokenized_sentences']  # Extract the tokenized sentences from note_info
        # Loop through each sentence_tokens list in the tokenized_sentences list
        for sentence_tokens in tokenized_sentences:
            print(' '.join(sentence_tokens))  # Print the tokens of each sentence separated by spaces
        print()  # Print an empty line after printing all sentences


Streaming output truncated to the last 5000 lines.
You were admitted with blood in the stool .
This may have been due to bleeding from a diverticulosis , or outpouching of the colon .
However , no definitive source of bleeding was found .
Please continue to take a new medication called lansoprazole to help prevent inflammation of the stomach .
Please have your blood count re - checked weekly while at the nursing facility to monitor for signs of dropping blood counts .
Please also have your primary care doctor assist you in scheduling outpatient follow - up with a gastroenterologist .
You may benefit from outpatient capsule endoscopy to try to find a source of your bleeding .
Take all medications as prescribed .
You should NOT take aggrenox for at least 2 weeks , furosemide ( also called lasix ) or metoprolol ( also called lopressor ) .
Please discuss restarting these medications with your primary care doctor .
Please also do not restart glucotrol .
Your blood sugars were low while in t